In [1]:
import pandas as pd
from flow_package.binary_flow_env import InputType, BinaryFlowEnv

FILE_PATH = "../data_cicids2017/9_test/friday.csv"
df = pd.read_csv(FILE_PATH)

test_normalized = pd.read_csv("../data_cicids2017/9_test/normalized_friday.csv")

In [2]:
train_input = InputType(
    data=df,
    sample_size=5000,
    normalize_exclude_columns=["Protocol", "Destination Port"],
    exclude_columns=["Attempted Category"]
)
train_env = BinaryFlowEnv(train_input)

initial_state = train_env.reset()
print(initial_state)

for i in range(10):
    action = train_env.action_space.sample()
    next_state, reward, done, truncated, info = train_env.step(action)
    print(next_state, reward, done, truncated, info)

[1.19800000e+03 6.00000000e+00 5.00007404e-07 0.00000000e+00
 5.19399574e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.63934277e-02 9.09101061e-07 0.00000000e+00 5.05194639e-07
 9.24252731e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.64785335e-04 7.31743012e-04 1.63934261e-02 1.63934426e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.66666667e-01 0.00000000e+00 3.09434663e-05
 2.00000000e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.56252384e-02 0.00000000e+00
 0.00000000e+00 6.00000000e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000